In [7]:
# 1️⃣ 구글 드라이브 마운트

from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/MyDrive/likelion/project2/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/likelion/project2


In [8]:
!pip install -U langchain-community
!pip install faiss-gpu
!pip install faiss-cpu
!pip install -U bitsandbytes

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [19]:
# ============================================================
# ✅ FAISS + Llama2 결합하여 RAG 질의응답 실행 (Score Mapping 포함)
# ============================================================
import os
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# 1️⃣ FAISS 인덱스 로드
faiss_path = "./project_2/data/db"
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

db = FAISS.load_local(
    faiss_path,
    embeddings,
    allow_dangerous_deserialization=True
)

print("✅ FAISS 인덱스 로드 완료!")


# ✅ 1-1. 스코어 매핑 + 필터링 래퍼 함수 추가
def retrieve_with_mapped_scores(db, query, k=3, threshold=0.5):
    """
    cosine score [-1,1] → [0,1] 매핑 후 threshold 이상만 반환
    """
    docs_and_scores = db.similarity_search_with_relevance_scores(query, k=k)
    filtered_docs = []

    for doc, score in docs_and_scores:
        mapped = (float(score) + 1.0) / 2.0  # [-1,1] → [0,1]
        if mapped >= threshold:
            filtered_docs.append(doc)

    return filtered_docs


# 2️⃣ Llama2 모델 로드
final_path = "./weight/output_rlhf/llama2/final_model"
MODEL_NAME = "beomi/llama-2-ko-7b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    final_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 3️⃣ HF Pipeline 생성
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=128,
    do_sample=False,
    top_p=0.9,
    temperature=0.7,
)

llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Llama2 모델 로드 완료!")

# 4️⃣ 프롬프트 템플릿 설정
prompt_template = """
#Preset
당신은 신뢰할 수 있는 전문가 어시스턴트입니다.
질문에 대해 아래 문맥(context)을 바탕으로 구체적이고 정확하게 답변하세요.
문맥에 관련 정보가 없거나 불충분할 경우, 당신의 일반 지식과 논리적 추론을 통해 답변하세요.
추측은 피하고, 확실하지 않은 경우 명확하게 언급하세요.

#RULE
핵심 정보를 간결히 요약하세요.
불필요한 서두나 반복을 피하세요.
가능하다면 근거를 간단히 제시하세요.
문체는 자연스럽고 전문가답게 유지하세요.

#Context:
{context}

#Question:
{question}

#Answer:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# ✅ 문서 포맷팅 함수 (기존 유지)
def format_docs_with_source(docs):
    if not docs:
        return "제공된 문맥 없음."
    formatted = []
    for doc in docs:
        meta_info = []
        if "title" in doc.metadata:
            meta_info.append(f"Title: {doc.metadata['title']}")
        if "source" in doc.metadata:
            meta_info.append(f"Source: {doc.metadata['source']}")
        if "year" in doc.metadata:
            meta_info.append(f"Year: {doc.metadata['year']}")
        if "institution" in doc.metadata:
            meta_info.append(f"Institution: {doc.metadata['institution']}")
        meta_str = " | ".join(meta_info)
        formatted.append(f"{doc.page_content}\n[{meta_str}]")
    return "\n\n".join(formatted)


# ✅ retriever 대체 함수 (Runnable 구조 깨지 않음)
def custom_retriever(query):
    docs = retrieve_with_mapped_scores(db, query, k=3, threshold=0.7)
    return format_docs_with_source(docs)


# RunnableMap / RunnableParallel 그대로 유지
prompt_runnable = RunnableMap(
    {
        "context": RunnablePassthrough() | custom_retriever,
        "question": RunnablePassthrough()
    }
) | prompt | llm | StrOutputParser()

full_chain = RunnableParallel({
    "question": RunnablePassthrough(),
    "context": RunnablePassthrough() | custom_retriever,
    "answer": prompt_runnable
})

print("✅ RAG 질의응답 체인 준비 완료!\n")


✅ FAISS 인덱스 로드 완료!


Device set to use cuda:0


✅ Llama2 모델 로드 완료!
✅ RAG 질의응답 체인 준비 완료!



# 코사인 벨런싱 적용 전

In [4]:
# ============================================================
# ✅ FAISS + Llama2 결합하여 RAG 질의응답 실행 (최신 LangChain 기준, OpenAI 없이 HuggingFaceEmbeddings)
# ============================================================
import os
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # ✅ OpenAI → HF 임베딩
# from langchain.chains import RetrievalQA # Removed this import
from langchain.llms import HuggingFacePipeline
# from langchain.prompts import PromptTemplate # Removed this import
# Import necessary components from their new locations
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate # Corrected import

# 1️⃣ FAISS 인덱스 로드
faiss_path = "./project_2/data/db"
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask", model_kwargs={"device": "cuda"}, encode_kwargs={"normalize_embeddings": True})

db = FAISS.load_local(
    faiss_path,
    embeddings,
    allow_dangerous_deserialization=True
)
# retriever = db.as_retriever(search_kwargs={"k": 3})  # 상위 3개 문서 검색
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5, "k": 3}) # 유사도 임계값 설정

print("✅ FAISS 인덱스 로드 완료!")

# 2️⃣ Llama2 LoRA 병합 모델 로드
final_path = "./weight/output_rlhf/llama2/final_model"
MODEL_NAME = "beomi/llama-2-ko-7b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    final_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 3️⃣ HF Pipeline 생성
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=128,
    do_sample=False,
    top_p=0.9,
    temperature=0.7,
)

llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Llama2 모델 로드 완료!")

# 4️⃣ 프롬프트 템플릿 설정
# 프롬프트 템플릿을 간결하게 수정하고, 문맥이 없을 경우에도 자연스럽도록 조정
prompt_template = """
#Preset
당신은 신뢰할 수 있는 전문가 어시스턴트입니다.
질문에 대해 아래 문맥(context)을 바탕으로 구체적이고 정확하게 답변하세요.
문맥에 관련 정보가 없거나 불충분할 경우, 당신의 일반 지식과 논리적 추론을 통해 답변하세요.
추측은 피하고, 확실하지 않은 경우 명확하게 언급하세요.

#RULE
핵심 정보를 간결히 요약하세요.
불필요한 서두나 반복을 피하세요.
가능하다면 근거를 간단히 제시하세요.
문체는 자연스럽고 전문가답게 유지하세요.

#Context:
{context}

#Question:
{question}

#Answer:
"""


prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Helper function to format retrieved documents or provide empty context
# Helper function: 문서 내용 + 출처 표시
def format_docs_with_source(docs):
    if not docs:
        return "제공된 문맥 없음."
    formatted = []
    for doc in docs:
        meta_info = []
        if "title" in doc.metadata:
            meta_info.append(f"Title: {doc.metadata['title']}")
        if "source" in doc.metadata:
            meta_info.append(f"Source: {doc.metadata['source']}")
        if "year" in doc.metadata:
            meta_info.append(f"Year: {doc.metadata['year']}")
        if "institution" in doc.metadata:
            meta_info.append(f"Institution: {doc.metadata['institution']}")
        meta_str = " | ".join(meta_info)
        formatted.append(f"{doc.page_content}\n[{meta_str}]")
    return "\n\n".join(formatted)

from langchain_core.runnables import RunnableMap

# context + question을 mapping으로 만들어 prompt에 전달
prompt_runnable = RunnableMap(
    {
        "context": retriever | format_docs_with_source,
        "question": RunnablePassthrough()
    }
) | prompt | llm | StrOutputParser()

# RunnableParallel 구성
full_chain = RunnableParallel({
    "question": RunnablePassthrough(),
    "context": retriever | format_docs_with_source,
    "answer": prompt_runnable
})




print("✅ RAG 질의응답 체인 준비 완료!\n")

✅ FAISS 인덱스 로드 완료!


Device set to use cuda:0


✅ Llama2 모델 로드 완료!
✅ RAG 질의응답 체인 준비 완료!



# 테스트

In [20]:
# 6️⃣ 테스트 질의
query = "숭례문은 언제 건축되었나요?"
# Use the full_chain to get the detailed result
result = full_chain.invoke(query) # Use invoke instead of run

print(result["answer"])


#Preset
당신은 신뢰할 수 있는 전문가 어시스턴트입니다. 
질문에 대해 아래 문맥(context)을 바탕으로 구체적이고 정확하게 답변하세요. 
문맥에 관련 정보가 없거나 불충분할 경우, 당신의 일반 지식과 논리적 추론을 통해 답변하세요. 
추측은 피하고, 확실하지 않은 경우 명확하게 언급하세요. 
 
#RULE
핵심 정보를 간결히 요약하세요.
불필요한 서두나 반복을 피하세요. 
가능하다면 근거를 간단히 제시하세요. 
문체는 자연스럽고 전문가답게 유지하세요.

#Context:
그 UN 로터리 쪽에서 문화회관 간판 그거 딱 보고 지나가면 문화회관이 있다고 생각했는데 순간적으로 지나가면 있죠, 문화회관이 사라져버리는 겁니다
이런 상태에서는 정문 활성화가 안 되거든요. 그런데 그거를 걷어내려면 돈도 많이 들 겁니다. 걷어내고 다시 옛날처럼 해 가지고 밑에서 올라가게 만들고 정문을 정문답게 만들고 하면.
그럼 저는 애초에 이걸, 몇 년도에 이걸 정문을 없애, 정문을 그렇게 만들었는지 모르겠지만 그때 정책적 판단을 누가 했는지 모르겠지만 지금 돌이켜 보면, 굉장히 잘못된 판단이다 이런 생각을 하고 있거든요. 그래서 완전히 그 건물을 완전히 죽여놨다.
그래서 지금 부산문화회관이 더 활성화 안 된 이유도 거기에 있다고 저는 생각을 하고 있습니다. 그래서 지금 <NAME> 대표이사께서 문화회관 정문을 활성화를 하겠다고 지금 나와 있기 때문에 이걸 이런 형태로 해 가지고 활성화해 가지고 안 됩니다, 이게.
그래서 그걸 걷어낼 건지 안 걷어낼 건지 그거부터 판단을 하셔 가지고 만약에 걷어낸다면 어떻게 그 앞에 그 단차가 제법 나거든요. 단차가 나기 때문에 그 단차 극복은 어떻게 할 건지 그런 계획을 주도면밀하게 해야 되고 그렇게 하려면 예산이 많이 들어갑니다, 이게.
지금 권 위원님이 주신 말씀 하고 저하고 너무 똑같아 갖고요. 제가 한 번 부산역에서 택시 타고 택시운전사 보고 그냥 아무 소리 안 하고 부산문화회관 정문으로 좀 데려다주십시오, 했더니 거기에 와서 뱅뱅 돌

In [21]:

# 6️⃣ 테스트 질의
query = "5·18민주유공자예우에 관한 법률은 무엇을 위한 목적을 가지고 있는가?"
# Use the full_chain to get the detailed result
result = full_chain.invoke(query) # Use invoke instead of run

print(result["answer"])


#Preset
당신은 신뢰할 수 있는 전문가 어시스턴트입니다. 
질문에 대해 아래 문맥(context)을 바탕으로 구체적이고 정확하게 답변하세요. 
문맥에 관련 정보가 없거나 불충분할 경우, 당신의 일반 지식과 논리적 추론을 통해 답변하세요. 
추측은 피하고, 확실하지 않은 경우 명확하게 언급하세요. 
 
#RULE
핵심 정보를 간결히 요약하세요.
불필요한 서두나 반복을 피하세요. 
가능하다면 근거를 간단히 제시하세요. 
문체는 자연스럽고 전문가답게 유지하세요.

#Context:
어느 정도 선이 적정할 것이고 어느 정도로 해 가지고 지원할 것이고 지원 목적은 어디까지 해 가지고 6조에다가 명문화시켜 놓은 상태…
하여튼 그런 부분들은 잘 검토를 하셔 가지고 효율적인 조례에 대해서 효율성이 있도록 그리해 주시기 바라고요
그다음에 공무원 복무 조례 좀 전에 우리 <NAME> 위원님께서도 잠시 질의한 7조 부분 있죠? 7조 부분에 5항이 조례에서 규칙으로 바뀌었네요, 그렇죠?
조례하고 규칙하고 법률적인 효력이 어느 게 큽니까?
조례가 큰데 이거를 결과적으로 물론 이걸 지키고 안 지키고 한다. 규칙이라서 안 지키고 이런 거는 아니겠지만 이거를 조례에 규정돼 있는 거를 조금 더 법률적인 효력이 더 큰 것을 작은 규칙으로 전환을 시키는 그 근본적인 원인은 뭡니까?
조례에서는, 아주 세부적인 사항을 예를 들어서 여성 모성보호라든지 임신부, 유아라든지 해 가지고 아주 세부적인 사항에 대해서는 조금 유연하게 하기 위해서 규칙으로 지금 저희들은 제정을 하고 있습니다, 조례에서. 조례에서 큰 묶음은 이 사항은 규칙으로 정한다 해 놓고 규칙의 세부적인 사항을 유연하게 하기 위해서 저희들 규칙으로.
그래서 어쨌든 이 조례로 정하는 게 조금 더 어떤 지키는 범위로 봐서는 좀 더 효력이 더 크지 않느냐 하는데 규칙으로 정한 거에 대한 의문점이 있어서 질의를 드렸고요.
그다음 15조 특별 휴가, 특별 휴가 이것도 보면 임의규정에서 강행규정으로 바꿔 놨다, 그렇죠?
이게 저희들이 보통 보

In [16]:
# 6️⃣ 테스트 질의
query = "텐동이 뭐야?"
# Use the full_chain to get the detailed result
result = full_chain.invoke(query) # Use invoke instead of run

print(result["answer"])


#Preset
당신은 신뢰할 수 있는 전문가 어시스턴트입니다. 
질문에 대해 아래 문맥(context)을 바탕으로 구체적이고 정확하게 답변하세요. 
문맥에 관련 정보가 없거나 불충분할 경우, 당신의 일반 지식과 논리적 추론을 통해 답변하세요. 
추측은 피하고, 확실하지 않은 경우 명확하게 언급하세요. 
 
#RULE
핵심 정보를 간결히 요약하세요.
불필요한 서두나 반복을 피하세요. 
가능하다면 근거를 간단히 제시하세요. 
문체는 자연스럽고 전문가답게 유지하세요.

#Context:
제공된 문맥 없음.

#Question:
텐동이 뭐야?

#Answer:
텐동은 일본식 튀김요리입니다. 
튀김옷을 입힌 재료를 기름에 튀겨낸 후, 소스를 곁들여 먹는 요리입니다. 
텐동은 주로 새우, 오징어, 고기, 채소 등을 재료로 사용합니다. 
텐동은 일본식 튀김요리인 덴푸라(天ぷら)의 한국식 발음입니다. 
텐동은 튀김옷을 입힌 재료를 기름에 튀겨낸 후, 소스를 곁들여 먹는 요리입니다. ​텐동은 주로 새우, 오징어, 고기, 채소


In [ ]:
import os

faiss_path = "./project_2/data/db" # FAISS 인덱스 경로

if os.path.exists(faiss_path):
    print(f"'{faiss_path}' 디렉토리 내용:")
    for item in os.listdir(faiss_path):
        item_path = os.path.join(faiss_path, item)
        if os.path.isfile(item_path):
            size_in_bytes = os.path.getsize(item_path)
            size_in_kb = size_in_bytes / 1024
            size_in_mb = size_in_bytes / (1024 * 1024)
            if size_in_mb > 1:
                 print(f"- {item}: {size_in_mb:.2f} MB")
            else:
                 print(f"- {item}: {size_in_kb:.2f} KB")

else:
    print(f"오류: '{faiss_path}' 디렉토리가 존재하지 않습니다.")

# keep

In [ ]:
# ============================================================
# ✅ FAISS + Llama2 결합하여 RAG 질의응답 실행 (최신 LangChain 기준, OpenAI 없이 HuggingFaceEmbeddings)
# ============================================================
import os
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # ✅ OpenAI → HF 임베딩
# from langchain.chains import RetrievalQA # Removed this import
from langchain.llms import HuggingFacePipeline
# from langchain.prompts import PromptTemplate # Removed this import
# Import necessary components from their new locations
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate # Corrected import
from langchain_core.documents import Document # Import Document class

# 1️⃣ FAISS 인덱스 로드
faiss_path = "./project_2/data/db"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = FAISS.load_local(
    faiss_path,
    embeddings,
    allow_dangerous_deserialization=True
)
# We will handle retrieval and filtering manually in the chain
# retriever = db.as_retriever(search_kwargs={"k": 3})
# retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8})

print("✅ FAISS 인덱스 로드 완료!")

# 2️⃣ Llama2 LoRA 병합 모델 로드
final_path = "./weight/output_rlhf/llama2/final_model"
MODEL_NAME = "beomi/llama-2-ko-7b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    final_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 3️⃣ HF Pipeline 생성
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=256,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
)

llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Llama2 모델 로드 완료!")

# 4️⃣ 프롬프트 템플릿 설정
# 프롬프트 템플릿을 간결하게 수정하고, 문맥이 없을 경우에도 자연스럽도록 조정
prompt_template = """주어진 문맥을 사용하여 질문에 답변하세요. 문맥에 정보가 없으면 가지고 있는 지식으로 답변하세요.

[문맥]
{context}

[질문]
{question}

[답변]
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Helper function to format retrieved documents or provide empty context
def format_docs(docs):
    if not docs:
        return "제공된 문맥 없음." # 검색 결과가 없을 때 더 명확한 placeholder 제공
    return "\n\n".join(doc.page_content for doc in docs)

# Custom retrieval and filtering function
def retrieve_and_filter_docs(query, db, k=5, score_threshold=0.7):
    # Get top k documents with scores
    docs_with_scores = db.similarity_search_with_score(query, k=k)

    # Filter documents based on score threshold
    filtered_docs = [doc for doc, score in docs_with_scores if score >= score_threshold]

    # Return Document objects without scores for compatibility with format_docs
    return filtered_docs

# 5️⃣ RAG 체인 구성 - Updated for LangChain v0.2.0+
# Create a chain that returns a dictionary with question, context, and answer
full_chain = RunnableParallel({
    "question": RunnablePassthrough(),
    "context": RunnablePassthrough() | (lambda query: retrieve_and_filter_docs(query, db, k=5, score_threshold=0.7)) | format_docs,
    "answer": (
        {
            "context": RunnablePassthrough() | (lambda query: retrieve_and_filter_docs(query, db, k=5, score_threshold=0.7)) | format_docs,
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
        | StrOutputParser()
    )
})


print("✅ RAG 질의응답 체인 준비 완료!\n")

# 6️⃣ 테스트 질의
query = "숭례문은 언제 건축되었나요?"
# Use the full_chain to get the detailed result
result = full_chain.invoke(query) # Use invoke instead of run

print("💬 질문:", result["question"])
print("📖 문맥:", result["context"])
print("💡 답변:", result["answer"])